In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Convert data from str and int to categorical
for col in train.columns:
    train[col] = pd.Categorical(train[col])
for col in test.columns:
    test[col] = pd.Categorical(test[col])

X_train = train.iloc[:, 1:]
X_test = test.iloc[:, 1:]
y_train = train['food_class']
y_test = test['food_class']

In [4]:
%%capture

features = ['food_class_l1', 'food_class_l2', 'food_class_l3', 'food_class_l4',
            'meal', 'semester', 'week_of_sem', 'dow', 'breakfast_or_not']

cboost = CatBoostClassifier(iterations=200,
                            learning_rate=0.01,
                            depth=6,
                            loss_function='MultiClass',
                            eval_metric='Accuracy',
                            random_seed=42)
cboost.fit(X_train, y_train, cat_features=features)

cboost_pred = cboost.predict(X_test)

In [5]:
accuracy_score(y_test, cboost_pred)

0.7567567567567568

In [6]:
confusion_matrix(y_test, cboost_pred)

array([[11,  7,  0],
       [ 2, 41,  0],
       [ 3,  6,  4]], dtype=int64)